In [8]:
import sys, tweepy
import os
import pickle

In [48]:
def twitter_auth():
    try:
        consumer_key = os.environ['TWITTER_API_KEY']
        consumer_secret = os.environ['TWITTER_API_SECRET']
        access_token = os.environ['TWITTER_API_ACCESS_TOKEN']
        access_secret=os.environ['TWITTER_API_ACCESS_TOKEN_SECRET']
    except KeyError:
        sys.stderr.write("environment variable not set\n")
        sys.exit(1)
    
    auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
    auth.set_access_token(access_token, access_secret)
    
    return auth

In [49]:
def get_twitter_client():
    auth = twitter_auth()
    client = tweepy.API(auth, wait_on_rate_limit=True)
    return client

In [50]:
def get_user_tweets(client, user, tweet_count=100):
    
    tweets = []
    
    for status in tweepy.Cursor(client.home_timeline, screen_name=user).items(tweet_count):
        tweets.append(status.text)
    
    return tweets

In [19]:
def get_users(filename=None):
    
    user_list = []
    
    if not filename:
        return None
    
    
    with open(filename) as f:
        for line in f:
            user_list.append(line.strip())
    
    return user_list

In [51]:
users = get_users('twitter_users_list')
tweets = []
client = get_twitter_client()
print(users)


['financialjuice', 'zerohedge', 'NorthmanTrader', 'caroljsroth', 'LynAldenContact', 'ReutersBiz']


In [52]:
for user in users:
    tweets.append(get_user_tweets(client, user, 100))

Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Rate limit reached. Sleeping for: 896
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected parameter: screen_name
Unexpected

In [90]:
tweets[4][7]

'Factbox: Some of the biggest splits in Corporate America https://t.co/uqK00WkkLl https://t.co/aceaktW6lH'

In [86]:
# pickle tweets for later use instead of hitting the API all the time

for i, u in enumerate(get_users('twitter_users_list')):
    with open('tweets/'+ u + '.txt', 'wb') as file:
        pickle.dump(tweets[i], file)


In [87]:
# Load pickled files

data = {}

for i, u in enumerate(users):
    with open("tweets/" + u + ".txt", "rb") as file:
        data[u] = pickle.load(file)
        

In [88]:
data.keys()

dict_keys(['financialjuice', 'zerohedge', 'NorthmanTrader', 'caroljsroth', 'LynAldenContact', 'ReutersBiz'])

In [95]:
# data.keys()
# print(data['LynAldenContact'])

In [112]:
len(data['financialjuice'])


100

In [44]:
# Clearn Data

In [91]:
def combine_tweet(list_of_tweets):
    # take list of tweets and combine it into one string
    combined_tweet = ' '.join(list_of_tweets)
    return combined_tweet

In [113]:
data_combined = {key: [combine_tweet(value)] for (key, value) in data.items()}
len(data_combined['financialjuice'])

1

In [94]:
import pandas as pd
pd.set_option('max_colwidth', 150)
data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['tweets']
data_df = data_df.sort_index()
data_df # raw corpus

,tweets
LynAldenContact,"Crypto mining company Bitfarms has made an about-face on its holding strategy and sold 3,000 Bitcoin for $62 millio… https://t.co/g2JGOtLQin Chevr..."
NorthmanTrader,"Elon Musk: “A recession is inevitable at some point. As to whether there is a recession in the near term, that is m… https://t.co/B6bikiD9AY Togo ..."
ReutersBiz,"The Best Neck Creams To Fight Sagging Skin, According To Dermatologists https://t.co/30dgs3lCw7 https://t.co/sKRUjUCuJr Apple is about to launch a..."
caroljsroth,"Crypto mining company Bitfarms has made an about-face on its holding strategy and sold 3,000 Bitcoin for $62 millio… https://t.co/g2JGOtLQin Chevr..."
financialjuice,"Elon Musk: “A recession is inevitable at some point. As to whether there is a recession in the near term, that is m… https://t.co/B6bikiD9AY Togo ..."
zerohedge,"Elon Musk: “A recession is inevitable at some point. As to whether there is a recession in the near term, that is m… https://t.co/B6bikiD9AY Togo ..."


In [99]:
#Apply a first roud of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase; remove punctuation'''
    text = text.lower()
    text = re.sub('\[.*?\]','', text)
    text = re.sub('["%s"]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub(r'http\S+', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [100]:
data_clean = pd.DataFrame(data_df.tweets.apply(round1))
data_clean

,tweets
LynAldenContact,crypto mining company bitfarms has made an aboutface on its holding strategy and sold bitcoin for millio… chevron el segundo could shut coker d...
NorthmanTrader,elon musk “a recession is inevitable at some point as to whether there is a recession in the near term that is m… togo is taking steps to substit...
ReutersBiz,the best neck creams to fight sagging skin according to dermatologists apple is about to launch a new method of proving you aren’t a bot in its ...
caroljsroth,crypto mining company bitfarms has made an aboutface on its holding strategy and sold bitcoin for millio… chevron el segundo could shut coker d...
financialjuice,elon musk “a recession is inevitable at some point as to whether there is a recession in the near term that is m… togo is taking steps to substit...
zerohedge,elon musk “a recession is inevitable at some point as to whether there is a recession in the near term that is m… togo is taking steps to substit...


In [101]:
# Pickle df for later use
data_df.to_pickle("corpus.pkl")

In [114]:
# Create document-term matrix using CountVectorizer
type(data_clean.tweets)

pandas.core.series.Series

In [104]:
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer(stop_words='english')
data_cv = cv.fit_transform(data_clean.tweets)
data_dtm = pd.DataFrame(data_cv.toarray(), columns=cv.get_feature_names())
data_dtm.index = data_clean.index
data_dtm

/Users/timal/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,abject,aboutface,accelerating,accepts,according,accounts,accused,achieve,act,actions,...,workfromhome,world,worlds,worrying,wowofortwayne,wyoming,yardeni,years,zelensky,zero
LynAldenContact,3,1,1,1,1,1,2,1,1,1,...,1,1,1,1,1,0,1,3,1,3
NorthmanTrader,1,0,1,1,1,1,2,1,1,1,...,1,1,1,1,1,0,1,3,1,3
ReutersBiz,3,1,1,1,2,1,2,1,0,1,...,1,1,1,1,1,1,0,1,0,3
caroljsroth,3,1,1,1,1,1,2,1,1,1,...,1,1,1,1,1,0,1,3,1,3
financialjuice,1,0,1,1,1,1,2,1,1,1,...,1,1,1,1,1,0,1,3,1,3
zerohedge,1,0,1,1,1,1,2,1,1,1,...,1,1,1,1,1,0,1,3,1,3


In [105]:
data_dtm.to_pickle("dtm.pkl")

In [106]:
data_clean.to_pickle('data_clean.pkl')
pickle.dump(cv, open('cv.pkl', 'wb'))